スクレイピングを用いて、ホテルおよび旅館の名前、評価、値段を取得

In [41]:
import requests
from bs4 import BeautifulSoup
import time

# スクレイピング関数
def scrape_accommodations(url):
    response = requests.get(url)
    
    # エンコーディングを明示的に設定
    response.encoding = 'Windows-31J'  
    soup = BeautifulSoup(response.content, 'html.parser')  # バイナリデータを使用

    # 必要なデータを格納するリスト
    accommodations = []

    # 宿泊施設情報を取得
    for item in soup.select('.p-searchResultItem'):  # 各宿泊施設の要素を取得
        try:
            name = item.select_one('.p-searchResultItem__facilityName').get_text(strip=True)
            stars = float(item.select_one('.p-searchResultItem__summaryaverage-num').get_text(strip=True))  # 星数
            price = item.select_one('.p-searchResultItem__lowestPriceValue').get_text(strip=True)  # 値段
            accommodations.append({'name': name, 'stars': stars, 'price': price})
        except AttributeError:
            continue  # 情報が欠けている場合はスキップ
    return accommodations

# ホテルのデータを取得するURL
hotel_url = "https://www.jalan.net/090000/LRG_090200/?dateUndecided=1&stayCount=1&roomCount=1&adultNum=2&minPrice=0&maxPrice=999999&yadHb=1&distCd=01&kenCd=090000&lrgCd=090200&roomCrack=200000&careOnsen=1"

# データの取得
hotel_data = scrape_accommodations(hotel_url)

# URLリスト
urls = [
    "https://www.jalan.net/090000/LRG_090200/?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=0",
    "https://www.jalan.net/090000/LRG_090200/page2.html?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=30",
    "https://www.jalan.net/090000/LRG_090200/page3.html?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=60"
]

# データ取得
ryokan_data_all_pages = []
for url in urls:  # 各ページURLを順番に処理
    print(f"Scraping: {url}")  # 現在処理中のURLを表示
    ryokan_data = scrape_accommodations(url)
    ryokan_data_all_pages.extend(ryokan_data)  # 各ページのデータを結合
    time.sleep(1)
# 重複削除
full_ryokan_data = {entry['name']: entry for entry in ryokan_data_all_pages}.values()

# 結果を表示
print("ホテルデータ:", hotel_data)
print("旅館データ:", list(full_ryokan_data))



Scraping: https://www.jalan.net/090000/LRG_090200/?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=0
Scraping: https://www.jalan.net/090000/LRG_090200/page2.html?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=30
Scraping: https://www.jalan.net/090000/LRG_090200/page3.html?screenId=UWW1402&distCd=01&activeSort=0&mvTabFlg=1&stayYear=&stayMonth=&stayDay=&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&yadRk=1&careOnsen=1&roomCrack=200000&kenCd=090000&lrgCd=090200&vosFlg=6&idx=60
ホテルデータ: [{'name': '草津温泉\u3000ホテルヴィレッジ', 'stars': 4.4, 'price': '19,800円〜'}, {'name': 'ラビスタ草津ヒルズ（共立リゾート）', 'stars': 4.6, 'price': '43,000円〜'}, {'name': '草津温泉\u3000ホテルヴィレッジ', 'stars': 4.4, 'price':

In [ ]:
それぞれのデータベースを作成

In [42]:
import sqlite3

# ホテル用データベースを作成
def save_hotel_data(data):
    conn = sqlite3.connect("hotel_data.db")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS hotels (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        price TEXT,
        stars REAL
    )
    """)

    data_for_insertion = [(hotel["name"], hotel["price"], hotel["stars"]) for hotel in data]

    cursor.executemany("""
        INSERT INTO hotels (name, price, stars)
        VALUES (?, ?, ?)
    """, data_for_insertion)

    conn.commit()
    conn.close()

save_hotel_data(hotel_data)



In [43]:
# 旅館用データベースを作成
def save_ryokan_data(data):
    conn = sqlite3.connect("ryokan_data.db")
    cursor = conn.cursor()

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS ryokans (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        price TEXT,
        stars REAL
    )
    """)

    data_for_insertion = [(ryokan["name"], ryokan["price"], ryokan["stars"]) for ryokan in data]

    cursor.executemany("""
    INSERT INTO ryokans (name, price, stars)
    VALUES (?, ?, ?)
    """, data_for_insertion)

    conn.commit()
    conn.close()

save_ryokan_data(ryokan_data)


In [ ]:
取得したデータをデータベースに挿入

In [44]:
# データを挿入する関数
def insert_hotel_data(hotel_data):
    conn = sqlite3.connect("hotel_data.db")
    cursor = conn.cursor()

    for hotel in hotel_data:
        # すでに存在する場合は挿入しない（重複を防ぐ）
        cursor.execute('''
        INSERT OR IGNORE INTO hotels (name, stars, price) 
        VALUES (?, ?, ?)
        ''', (hotel['name'], hotel['stars'], hotel['price']))

    # コミットして変更を保存
    conn.commit()
    conn.close()

# データベースに挿入
insert_hotel_data(hotel_data)


In [45]:
# データを挿入する関数
def insert_ryokan_data(ryokan_data):
    conn = sqlite3.connect("ryokan_data.db")
    cursor = conn.cursor()

    for ryokan in ryokan_data:
        cursor.execute('''
        INSERT OR IGNORE INTO ryokans (name, stars, price) 
        VALUES (?, ?, ?)
        ''', (ryokan['name'], ryokan['stars'], ryokan['price']))

    conn.commit()
    conn.close()

insert_ryokan_data(full_ryokan_data)

In [ ]:
データベースの確認のためデータの表示

In [46]:
import pandas as pd

def load_hotel_data():
    conn = sqlite3.connect("hotel_data.db")
    query = "SELECT * FROM hotels"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

def load_ryokan_data():
    conn = sqlite3.connect("ryokan_data.db")
    query = "SELECT * FROM ryokans"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

def print_hotel_data():
    df = load_hotel_data()
    
    # 重複を削除（name列で重複を削除）
    df = df.drop_duplicates(subset='name', keep='first')
    
    print("Hotel Data:")
    print(df)  

def print_ryokan_data():
    df = load_ryokan_data()
    
    # 重複を削除（name列で重複を削除）
    df = df.drop_duplicates(subset='name', keep='first')
    
    print("Ryokan Data:")
    print(df)  

# ホテルと旅館のデータを表示
print_hotel_data()
print_ryokan_data()


Hotel Data:
    id                    name     price  stars
0    1           草津温泉　ホテルヴィレッジ  19,800円〜    4.4
1    2       ラビスタ草津ヒルズ（共立リゾート）  43,000円〜    4.6
2    3             草津ナウリゾートホテル  33,000円〜    4.1
4    5                草津温泉　VAN  66,480円〜    4.8
5    6             亀の井ホテル 草津湯畑  25,800円〜    4.1
6    7  フォートリート草津〈2023年4月オープン〉  19,800円〜    4.3
7    8               草津温泉　ふたばや  12,600円〜    4.8
8    9          リブマックスリゾート草津温泉  23,800円〜    4.0
9   10            草津温泉　千　〜ｓｅｎ〜  16,000円〜    4.2
10  11                アイソネット草津  19,800円〜    4.9
11  12            ビジネスホテル アゼリア  12,000円〜    3.9
Ryokan Data:
    id                 name     price  stars
0    1               湯宿　みゆき  20,900円〜    4.5
1    2         草津温泉　和える宿　高松  26,400円〜    4.1
2    3            草津温泉　湯畑草菴  22,000円〜    3.7
3    4       草津温泉３２６　山の湯ホテル  17,700円〜    4.2
4    5        いきつけの宿　旅館　田島屋   9,680円〜    4.2
..  ..                  ...       ...    ...
64  65                上村屋旅館  16,800円〜    4.5
65  66        草津温泉　ホテルニュー紅葉  11,660円〜  

In [47]:
import pandas as pd

# 例: hotel_data と ryokan_data がリスト形式で保存されている場合、データフレームに変換
hotel_data_df = pd.DataFrame(hotel_data)
ryokan_data_df = pd.DataFrame(ryokan_data)

def convert_price(price):
    # 「円〜」や「円～」、カンマを取り除き、数値に変換
    price = price.replace('円〜', '').replace('円～', '').replace(',', '')
    try:
        return int(price)
    except ValueError:
        return None  # 数値に変換できない場合は None を返す


# ホテルデータの平均を計算
hotel_data_df['price'] = hotel_data_df['price'].apply(convert_price)  # 価格を数値に変換
hotel_avg = hotel_data_df[['price', 'stars']].mean()

# 旅館データの平均を計算
ryokan_data_df['price'] = ryokan_data_df['price'].apply(convert_price)  # 価格を数値に変換
ryokan_avg = ryokan_data_df[['price', 'stars']].mean()

# 結果を表示
print(f"ホテルの平均: \n{hotel_avg}")
print(f"\n旅館の平均: \n{ryokan_avg}")


ホテルの平均: 
price    25506.666667
stars        4.375000
dtype: float64

旅館の平均: 
price    27284.615385
stars        4.507692
dtype: float64
